Jupyter Notebook voor het trainen van een objectdetectiemodel dat scheuren in het wegdek markeert.

Dit notebook maakt gebruik van Pytorch en Detectron2 (van Meta/Facebook). Om dit model te gebruiken is een grafische kaart (GPU) vereist.

Daarnaast wordt voor de opslag van de bestanden gebruik gemaakt van Google Drive vanwege het gebrek aan een grafische kaart tijdens het ontwikkelen. De opslaglocatie kan in de code worden veranderd naar wens op de aangegeven plekken.  

**Stap 1: Systeemconfiguratie en installatie van dependencies**

De volgende cellen installeren de benodigde software en configureren de omgeving.

In [1]:
!sudo apt-get update
!sudo apt-get install -y locales
!sudo locale-gen en_US.UTF-8
!sudo update-locale LANG=en_US.UTF-8
!export LANG=en_US.UTF-8

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,197 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,563 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://security.ubuntu.com/ubuntu jammy-sec

Installeren van benodigde packages. De Google Colab-omgeving heeft al veel packages geïnstalleerd, dus het kan zijn dat er in deze stap meer packages moeten worden geïnstalleerd als het script lokaal wordt gerund. Om een package te installeren, voeg een nieuwe regel code toe met de volgende tekst:

!pip install (package naar keuze)

In [2]:
# Install PyTorch and torchvision
!pip install -U torch torchvision

# Install Detectron2
!pip install git+https://github.com/facebookresearch/fvcore
!pip install 'git+https://github.com/facebookresearch/detectron2.git'

# Install ipywidgets
!pip install ipywidgets

  Cloning https://github.com/facebookresearch/fvcore to /tmp/pip-req-build-zgqh73gb
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/fvcore /tmp/pip-req-build-zgqh73gb
  Resolved https://github.com/facebookresearch/fvcore to commit a491d5b9a06746f387aca2f1f9c7c7f28e20bef9
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.6-py3-none-any.whl size=65670 sha256=86da57bb9b5040c32027d6e38250a627f97312ca32c6fcc69fbc3f4d7fd217b0
  Stored in directory: /tmp/pip-ephem-wheel-cache-4_igyfwn/wheels/a6/fe/66/6e6bfe16b94f13bf3e150e48ad58e6dc20de14125d0550621e
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31528 sha256=6221f2f83ad4a37d21d66ceeb6ab74b2d122e968f150e4b8d0b2471048cf4af2
  Stored in directory: /root/.cache/pip/wheels/9a/a3/b6/ac0fcd1b4ed5cfeb3db92e6a0e476cfd48ed0df9

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 27.6 MB/s eta 0:00:00


In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.5 MB/s eta 0:00:00


**Stap 2: Importeren van packages**

Het volgende celblok importeert alle packages die nodig zijn om het notebook te gebruiken. Als er packages missen, ga terug naar de volgende stap en controleer of deze correct is geïnstalleerd.

In [ ]:
import ipywidgets as widgets
from IPython.display import display
import os
import cv2
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
from google.colab.patches import cv2_imshow
import xml.etree.ElementTree as ET
import json
import re
from detectron2.data.datasets import register_coco_instances
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer
import matplotlib.pyplot as plt
from PIL import Image
from detectron2.utils.logger import setup_logger
import optuna


**Stap 3: Data voorbereiden**

In deze stap wordt de dataset gestructureerd en voorbereid voor gebruik.

Allereerst kiezen we het pad waar onze bestanden worden opgeslagen. Zoals eerder vermeld wordt hier gebruik gemaakt van Google Drive vanwege het gebrek aan een GPU.  


In [ ]:
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [ ]:
!ls /content/drive/MyDrive/R-CNN/test #Pas aan naar gewenste pad

Czech_000003.jpg  Czech_000074.jpg  India_000020.jpg  India_000091.jpg	Japan_000102.jpg
Czech_000005.jpg  Czech_000089.jpg  India_000033.jpg  India_000094.jpg	Japan_000107.jpg
Czech_000008.jpg  Czech_000129.jpg  India_000034.jpg  India_000137.jpg	Japan_000109.jpg
Czech_000023.jpg  Czech_000134.jpg  India_000045.jpg  Japan_000039.jpg	Japan_000116.jpg
Czech_000029.jpg  Czech_000136.jpg  India_000056.jpg  Japan_000048.jpg	Japan_000125.jpg
Czech_000036.jpg  Czech_000151.jpg  India_000058.jpg  Japan_000054.jpg	Japan_000132.jpg
Czech_000046.jpg  India_000004.jpg  India_000062.jpg  Japan_000055.jpg	Japan_000139.jpg
Czech_000049.jpg  India_000008.jpg  India_000068.jpg  Japan_000078.jpg	Japan_000156.jpg
Czech_000052.jpg  India_000013.jpg  India_000070.jpg  Japan_000083.jpg	Japan_000163.jpg


Datasetstructuur instellen

In [ ]:
# Define the base directory
base_dir = "/content/drive/MyDrive/R-CNN/train" #Pas aan naar gewenste pad
images_dir = os.path.join(base_dir, "images")
xmls_dir = os.path.join(base_dir, "xmls")
coco_dir = os.path.join(base_dir, "coco")

# Define the path to the label map
label_map_path = "/content/drive/MyDrive/R-CNN/train/label_map.pbtxt" #Pas aan naar gewenste pad


Labelmap parseren om de tekstlabels te koppelen aan numerieke waarden. Het model maakt gebruik van numerieke waarden in plaats van tekstlabels.

In [ ]:
# Extract labels and IDs from the label map
def parse_label_map(label_map_path):
    with open(label_map_path, 'r') as f:
        label_map_content = f.read()

    # Regex to extract ID and name pairs
    pattern = r"item\s*{\s*id:\s*(\d+)\s*name:\s*'([^']+)'\s*}"
    matches = re.findall(pattern, label_map_content)

    # Return a dictionary of {id: name}
    return {int(match[0]): match[1] for match in matches}

# Parse the label map
label_map = parse_label_map(label_map_path)
print(f"Parsed Label Map: {label_map}")


Parsed Label Map: {1: 'D00', 2: 'D01', 3: 'D10', 4: 'D11', 5: 'D20', 6: 'D40', 7: 'D43', 8: 'D44', 9: 'D50'}


Nu worden de labels geconverteerd naar het COCO-format. Het COCO-format wordt vaak gebruikt in objectdetectie en beeldsegmentatie vanwege de gestandaardiseerde structuur en de vele mogelijkheden die het biedt voor het representeren van datasets.

In [ ]:
def convert_to_coco_format_with_label_map(base_dir, label_map):
    # Define centralized directories
    images_dir = os.path.join(base_dir, "images")
    xmls_dir = os.path.join(base_dir, "xmls")
    coco_dir = os.path.join(base_dir, "coco")
    os.makedirs(coco_dir, exist_ok=True)

    # Prepare COCO data structure
    data = {
        "images": [],
        "annotations": [],
        "categories": []
    }
    annotation_id = 1
    image_id = 1

    # Add category information from label_map
    for category_id, category_name in label_map.items():
        data["categories"].append({
            "id": category_id,
            "name": category_name
        })

    # Process each XML file in the centralized xmls directory
    for xml_file in os.listdir(xmls_dir):
        if not xml_file.endswith(".xml"):
            continue

        tree = ET.parse(os.path.join(xmls_dir, xml_file))
        root = tree.getroot()

        filename = root.find('filename').text
        width = int(root.find('size/width').text)
        height = int(root.find('size/height').text)

        # Ensure the corresponding image exists
        image_path = os.path.join(images_dir, filename)
        if not os.path.exists(image_path):
            print(f"Warning: Image {filename} not found for XML {xml_file}. Skipping.")
            continue

        # Add image information to the COCO dataset
        data["images"].append({
            "id": image_id,
            "file_name": filename,  # Path relative to images_dir
            "width": width,
            "height": height
        })

        # Extract bounding box details
        for obj in root.findall('object'):
            label = obj.find('name').text
            bbox = obj.find('bndbox')
            x_min = int(bbox.find('xmin').text)
            y_min = int(bbox.find('ymin').text)
            x_max = int(bbox.find('xmax').text)
            y_max = int(bbox.find('ymax').text)

            bbox_width = x_max - x_min
            bbox_height = y_max - y_min

            # Ensure the category exists in the label map
            category_id = next((id for id, name in label_map.items() if name == label), None)
            if category_id is None:
                continue  # Skip annotations with unknown categories

            data["annotations"].append({
                "id": annotation_id,
                "image_id": image_id,
                "category_id": category_id,
                "bbox": [x_min, y_min, bbox_width, bbox_height],
                "area": bbox_width * bbox_height,
                "iscrowd": 0
            })

            annotation_id += 1

        image_id += 1

    # Save the COCO JSON file
    coco_output_path = os.path.join(coco_dir, "coco_annotations.json")
    with open(coco_output_path, 'w') as f:
        json.dump(data, f)

    print(f"COCO annotations saved to {coco_output_path}")

# Convert to COCO format
convert_to_coco_format_with_label_map(base_dir, label_map)


COCO annotations saved to /content/drive/MyDrive/R-CNN/train/coco/coco_annotations.json


Nu wordt de dataset gesplitst in een trainings- en testset.

In [ ]:
from sklearn.model_selection import train_test_split

def split_coco_dataset(coco_annotations_path, split_ratio=0.8):
    # Load the COCO annotations
    with open(coco_annotations_path, 'r') as f:
        coco_data = json.load(f)

    # Split images into training and validation sets
    images = coco_data["images"]
    train_images, val_images = train_test_split(images, train_size=split_ratio, random_state=42)

    # Map image IDs for splitting annotations
    train_image_ids = {img["id"] for img in train_images}
    train_annotations = [ann for ann in coco_data["annotations"] if ann["image_id"] in train_image_ids]
    val_annotations = [ann for ann in coco_data["annotations"] if ann["image_id"] not in train_image_ids]

    # Save the split datasets
    train_data = {"images": train_images, "annotations": train_annotations, "categories": coco_data["categories"]}
    val_data = {"images": val_images, "annotations": val_annotations, "categories": coco_data["categories"]}

    train_path = os.path.join(coco_dir, "coco_train.json")
    val_path = os.path.join(coco_dir, "coco_val.json")

    with open(train_path, 'w') as f:
        json.dump(train_data, f)
    with open(val_path, 'w') as f:
        json.dump(val_data, f)

    return train_path, val_path

coco_annotations_path = os.path.join(base_dir, "coco", "coco_annotations.json")
# Split the dataset
train_annotations_path, val_annotations_path = split_coco_dataset(coco_annotations_path)


**Stap 4: Modelconfiguratie en registratie van datasets**

In deze stap wordt het Faster R-CNN-model ingesteld, en worden de trainings- en validatiesets geregistreerd in Detectron2.

Allereerst wordt de splitsing van de dataset gecontroleerd:

In [ ]:
with open(train_annotations_path, 'r') as f:
    train_data = json.load(f)
print("Number of training images:", len(train_data["images"]))

with open(val_annotations_path, 'r') as f:
    val_data = json.load(f)
print("Number of validation images:", len(val_data["images"]))


Number of training images: 168
Number of validation images: 42


Daarna worden de labels gecontroleerd.

In [ ]:
def extract_labels_from_pbtxt(label_map_path):
    with open(label_map_path, 'r') as f:
        label_map_content = f.read()

    # Use regex to extract all label names from label_map.pbtxt
    pattern = re.compile(r"name:\s*'(.+)'")
    labels = pattern.findall(label_map_content)
    return labels

# Define the path to label_map.pbtxt
label_map_path = "/content/drive/MyDrive/R-CNN/train/label_map.pbtxt"

# Extract labels
labels = extract_labels_from_pbtxt(label_map_path)
print(labels)  # Check if labels are correctly extracted


['D00', 'D01', 'D10', 'D11', 'D20', 'D40', 'D43', 'D44', 'D50']


In het volgende blok worden de trainings- en validatiesets geregistreerd in het model:

In [ ]:
# Register the training and validation datasets
register_coco_instances("crack_train_split", {}, train_annotations_path, os.path.join(base_dir, "images"))
register_coco_instances("crack_val_split", {}, val_annotations_path, os.path.join(base_dir, "images"))

print("Training and validation datasets registered successfully!")


Training and validation datasets registered successfully!


Nu worden de annotaties nogmaals gecontroleerd:

In [ ]:
# Load and inspect the training annotations
with open(train_annotations_path, 'r') as f:
    train_data = json.load(f)

print("Sample images from training annotations:")
for image in train_data['images'][:5]:  # Print the first 5 entries
    print(image['file_name'])

print("Sample annotations from training annotations:")
for annotation in train_data['annotations'][:5]:  # Print the first 5 entries
    print(annotation)


Sample images from training annotations:
Japan_000082.jpg
India_000103.jpg
Japan_000035.jpg
Japan_000069.jpg
Czech_000048.jpg
Sample annotations from training annotations:
{'id': 1, 'image_id': 2, 'category_id': 6, 'bbox': [268, 469, 59, 35], 'area': 2065, 'iscrowd': 0}
{'id': 2, 'image_id': 6, 'category_id': 6, 'bbox': [474, 570, 112, 51], 'area': 5712, 'iscrowd': 0}
{'id': 3, 'image_id': 6, 'category_id': 6, 'bbox': [478, 511, 80, 37], 'area': 2960, 'iscrowd': 0}
{'id': 4, 'image_id': 6, 'category_id': 6, 'bbox': [191, 470, 109, 77], 'area': 8393, 'iscrowd': 0}
{'id': 5, 'image_id': 6, 'category_id': 6, 'bbox': [1, 624, 167, 87], 'area': 14529, 'iscrowd': 0}


**Stap 5: Modeltraining**

Nu wordt het model getraind met behulp van Detectron2's DefaultTrainer.

In [ ]:
# Define the base directory
base_dir = "/content/drive/MyDrive/R-CNN/train" #Pas aan naar wens
images_dir = os.path.join(base_dir, "images")

# Paths for split annotations
train_annotations_path = os.path.join(base_dir, "coco", "coco_train.json")
val_annotations_path = os.path.join(base_dir, "coco", "coco_val.json")

# Register the split datasets
register_coco_instances("crack_train_split", {}, train_annotations_path, images_dir)
register_coco_instances("crack_val_split", {}, val_annotations_path, images_dir)

# Configure Detectron2
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))

# Update to use the split datasets
cfg.DATASETS.TRAIN = ("crack_train_split",)
cfg.DATASETS.TEST = ("crack_val_split",)  # Use validation set for evaluation
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 1000  # Pas aan naar wens
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(label_map)  # Number of classes from the label map

# Set the output directory
cfg.OUTPUT_DIR = os.path.join(base_dir, "output")
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

# Train the model
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()


[12/16 15:32:31 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

model_final_280758.pkl: 167MB [00:00, 203MB/s]                           
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}


[12/16 15:32:32 d2.engine.train_loop]: Starting training from iteration 0


/usr/local/lib/python3.10/dist-packages/torch/functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3595.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[12/16 15:32:56 d2.utils.events]:  eta: 0:14:22  iter: 19  total_loss: 2.607  loss_cls: 2.236  loss_box_reg: 0.02069  loss_rpn_cls: 0.3161  loss_rpn_loc: 0.02223    time: 0.9064  last_time: 1.3481  data_time: 0.6293  last_data_time: 1.1225   lr: 4.9953e-06  max_mem: 1736M
[12/16 15:33:16 d2.utils.events]:  eta: 0:14:06  iter: 39  total_loss: 2.491  loss_cls: 2.075  loss_box_reg: 0.001446  loss_rpn_cls: 0.3595  loss_rpn_loc: 0.02552    time: 0.8729  last_time: 0.9359  data_time: 0.5572  last_data_time: 0.6752   lr: 9.9902e-06  max_mem: 1736M
[12/16 15:33:26 d2.utils.events]:  eta: 0:13:18  iter: 59  total_loss: 2.378  loss_cls: 1.741  loss_box_reg: 0.002138  loss_rpn_cls: 0.4525  loss_rpn_loc: 0.04073    time: 0.7320  last_time: 0.2500  data_time: 0.1977  last_data_time: 0.0049   lr: 1.4985e-05  max_mem: 1736M
[12/16 15:33:32 d2.utils.events]:  eta: 0:10:07  iter: 79  total_loss: 1.631  loss_cls: 1.252  loss_box_reg: 0.002377  loss_rpn_cls: 0.2894  loss_rpn_loc: 0.02986    time: 0.6205 

**Stap 6: Modeloptimalisatie (optioneel)**

In deze stap kan het script hyperparameters optimaliseren met Optuna om mogelijk de nauwkeurigheid te verbeteren. Echter kost dit veel GPU-kracht, dus dit is uitgecommentarieerd om GPU-belasting te verminderen.

In [ ]:

""""
setup_logger()
def objective(trial):
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))

    # Use split datasets
    cfg.DATASETS.TRAIN = ("crack_train_split",)
    cfg.DATASETS.TEST = ("crack_val_split",)
    cfg.DATALOADER.NUM_WORKERS = 2

    # Sample hyperparameters
    cfg.SOLVER.BASE_LR = trial.suggest_loguniform("base_lr", 1e-5, 1e-3)
    cfg.SOLVER.IMS_PER_BATCH = trial.suggest_categorical("ims_per_batch", [2, 4, 8])
    cfg.SOLVER.MAX_ITER = trial.suggest_int("max_iter", 500, 3000, step=500)
    cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = trial.suggest_categorical("batch_size_per_image", [64, 128, 256])

    # Set number of classes and pre-trained weights
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(label_map)
    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")

    # Output directory for the trial
    cfg.OUTPUT_DIR = f"./output/trial_{trial.number}"

    # Train the model
    trainer = DefaultTrainer(cfg)
    trainer.resume_or_load(resume=False)
    trainer.train()

    # Evaluate the model on the validation set
    evaluator = COCOEvaluator("crack_val_split", cfg, False, output_dir=cfg.OUTPUT_DIR)
    val_loader = build_detection_test_loader(cfg, "crack_val_split")
    metrics = inference_on_dataset(trainer.model, val_loader, evaluator)

    # Return the validation mAP (bbox Average Precision)
    return metrics["bbox"]["AP"]

# Run Optuna optimization
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=5)

# Print the best hyperparameters
print("Best hyperparameters:", study.best_params)
"""

**Stap 7: Modelinferentie en interactieve visualisatie**

Nu wordt het model geladen en gebruikt voor inferentie. Vervolgens wordt een interactieve widget geladen om de resultaten te tonen.

In [ ]:
# Load the trained model for inference
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # Betrouwbaarheidsinterval kan hier worden aangepast
predictor = DefaultPredictor(cfg)


[12/16 15:38:42 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /content/drive/MyDrive/R-CNN/train/output/model_final.pth ...


/usr/local/lib/python3.10/dist-packages/fvcore/common/checkpoint.py:252: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, map_location=torch.device("cpu"))

In [ ]:
# Define the unified image folder path
images_dir = "/content/drive/MyDrive/R-CNN/test" #Pas aan naar wens

# Get a list of all images in the unified folder
image_files = [f for f in os.listdir(images_dir) if f.endswith(('.jpg', '.png'))]

# Create the dropdown menu for selecting an image
image_dropdown = widgets.Dropdown(
    options=image_files,
    value=image_files[0] if image_files else None,
    description="Select Image:",
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

# Placeholder for displaying the selected image and inference results
image_display = widgets.Output()

# Update the displayed image when an image is selected
def update_image_display(change):
    selected_image = change['new']
    if selected_image:
        # Path to the selected image
        image_path = os.path.join(images_dir, selected_image)

        # Display the selected image
        image_display.clear_output(wait=True)
        with image_display:
            img = Image.open(image_path)
            plt.imshow(img)
            plt.axis('off')
            plt.show()

# Set the handler for when the image selection changes
image_dropdown.observe(update_image_display, names='value')

# Perform inference on the selected image
def run_inference_on_selected_image(image_name):
    image_path = os.path.join(images_dir, image_name)

    # Load the selected image
    img = cv2.imread(image_path)

    # Perform predictions
    outputs = predictor(img)

    # Visualize and display results
    v = Visualizer(img[:, :, ::-1], metadata=MetadataCatalog.get("crack_train"), scale=0.5)
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))

    # Display the result
    image_display.clear_output(wait=True)
    with image_display:
        plt.imshow(out.get_image()[:, :, ::-1])
        plt.axis('off')
        plt.show()

# Create a button to trigger inference
run_button = widgets.Button(
    description='Run Inference',
    button_style='success',
    tooltip='Run inference on the selected image',
    icon='check'
)

# Define the action for the button click
def on_run_button_clicked(b):
    selected_image = image_dropdown.value
    if selected_image:
        run_inference_on_selected_image(selected_image)

# Attach the button click event to the handler
run_button.on_click(on_run_button_clicked)

# Layout the dropdown and display
display(widgets.VBox([image_dropdown, run_button, image_display]))
